## <p style="text-align: center;">Transformers: Attention is All You Need!</p>

### References:
- Nice explanation with illustration (Read fully and try to understand everything) - [link](https://www.michaelphi.com/illustrated-guide-to-transformers/)
- Another Nice explanation with examples - [link](http://jalammar.github.io/illustrated-transformer/)
- Transformers - [link](https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04)
- You can also refer to the official paper: <b>Attention is all you need!</b>
- Self Attention explained with example (illustration) - [link](https://towardsdatascience.com/illustrated-self-attention-2d627e33b20a)

- Attention and Transformers - [link](https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/)

- Stanford - Checkout first few slides (also try to search if the crresponding course video lecture is available_ - [link](https://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture14-transformers.pdf)

<i><b>NOTE : </b><font color='purple'>The code you wrote is the <b>most basic and simplest Transformer architecture ever</b>. Please refer to the <b>[Hugging Face library](https://huggingface.co/transformers/index.html)</b> for advanced and complex Transformers with pretrained weights. this library also provides you with an option of fine-tuning the models (i.e. you can use the pretrained weights as initial weights and train a model on your own dataset if required - I think its not necessary for beginners)</font>
<font color='darkblue'>
- Refer  <b>'USING 🤗 TRANSFORMERS'</b>  section for general tutorials on how to use the huggingface library.
</font>

## Introduction:
Transformers - Recurrent Networks were, until now, one of the best ways to capture the timely dependencies in sequences. However, the team presenting the paper proved that an architecture with only attention-mechanisms without any RNN (Recurrent Neural Networks) can improve on the results in translation task and other tasks!

- Simple NN - no context, no postion
- Simple LSTM encoder-decoder - captures context (difficult for longer sentences) and position
- Transformers - captures both context (not difficult for longer sentences) and position of words

<img style="float: left;" src="./assets/transformer3.png">

## 1. Positional Encoding:

- Since we are not using any RNNs we wont be able to capture the order/position of words in a sentence. In order to capture that we use positional encoding.
- <b>For a detailed explanation refer : [link](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/#what-is-positional-encoding-and-why-do-we-need-it-in-the-first-place) & [link](https://medium.com/swlh/elegant-intuitions-behind-positional-encodings-dc48b4a4a5d1)</b>
- Given a sequence of words, we process into word embeddings Zʷ: N x hʷ, N represents the number of words in a sampled sequence, hʷ represents the embedding size. Then, pos ∈ [0, N-1] is the position of the word in the sequence and i ∈ [0, hʷ-1] is the index which spans the dimension of the word embedding.


#### ..................................................................................................................................................................................................................................................
#### Given a word embeddings Zʷ: N x hʷ

<img style="float: left;" src="https://miro.medium.com/max/507/1*NBetROvAUpwf3KH31-nAOg.png">

            - N: Number of word in the sequence
            - hʷ: Dimension size of word embedding (d_model)
            - pos: position of the current word in the sequence in [0, N-1]
            - i: index of the dimensional index of word embedding in [0, hʷ-1]

#### ..................................................................................................................................................................................................................................................
#### For FAQs such as the following refer this [link](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/#what-is-positional-encoding-and-why-do-we-need-it-in-the-first-place) :

- Why positional embeddings are summed with word embeddings instead of concatenation?
- Doesn't the position information get vanished once it reaches the upper layers? (cant seem to undersatnd the question...)
- Why are both sine and cosine used?

# 2. Masking:

<b> Padding Mask:</b> used in all three Multi-head Attention blocks of the Transformers (since the inputs they get have padded values)
- Now that all samples have a uniform length, the model must be informed that some part of the data is actually padding and should be ignored.
- To ensure that this is done, we mask padded positions (setting them to `-inf`) before the softmax step in the self-attention calculation.
- When these raw attention vaules are passed through a softmax layer, attention values at masked positions become '0' since `e^(-inf)` = 0

<b> Look-ahead Mask:</b> used in the 2nd Multi-head Attention block of the Decoder layer 
- Lets say we have the following sequence as an input for our decoder: “I love it”, then the expected prediction for the token at position two (“love”) is the token at the next position (“it”). We do not want the attention mechanism to share any information regarding the third token, when giving a prediction for the second token.
- To ensure that this is done, we mask future positions (setting them to `-inf`) before the softmax step in the self-attention calculation.
- When these raw attention vaules are passed through a softmax layer, attention values at masked positions become '0' since `e^(-inf)` = 0
- the Transformer applies a mask to the input in the 1st Multi-head Attention module (of both Encoder and Decoder part) to avoid seeing potential ‘future’ sequence elements. <b>This is specific to the Transformer architecture because we do not have RNNs where we can input our sequence sequentially.</b> Here, we input everything together and if there were no mask, the multi-head attention would consider the whole decoder input sequence at each position.

# 3. Multi-head Attention:

<i>Lets say your model (Multi-head Attention) has 4 heads and the dimension of the input matrix is `7x512` (sequence length = 7, embedding dim = 512). Now the input matrix is split <b>(copied/cloned)</b> into 4 parts each of dimension `7x128` and then these 4 matrices are fed into each heads which then outputs vectors each of dimension `7x128`. These outputs are then concatenated to form a matrix of dimension `7x512` and then passed through a <b>Linear layer</b>.</i>

The type of attention that uses all the encoder hidden states is also known as <b><u>global attention</u>.</b> In contrast, <b><u>local attention</u></b> uses only a subset of the encoder hidden states. As the scope of this article (my notes) is global attention, any references made to “attention” in this article are taken to mean “global attention.”

    - Q is a matrix that contains the query (vector representation of one word in the sequence)
    - K are all the keys (vector representations of all the words in the sequence)
    - V are the values, which are again the vector representations of all the words in the sequence. 

For the 1st Multi-head Attention module (of the Encoder and Decoder part), V is identical to Q. However, for the attention module that is taking into account the encoder and the decoder sequences (i.e. 2nd Multi-head Attention module of the Decoder part), V is different from the sequence represented by Q.

### <u>Intuition behind Scaled dot product attention</u>:
- For calculating attention weights why do we start with the dot-product of `Query` and `Key`
- We know that `Query` is the vector representation of <b><u>one word in the sequence</u></b> and `Key` is the vector representations of <b><u>all the words in the sequence</u>.</b> So when you do dot-product of Query with each and every word of the Key, the resulting product (attention weights) will be nearly 1 for all those vectors of Keys which are identical to the Query vector.

### <i>What is the use of each <u>Multi-head Attention</u> (MHA) block and the <u>Attention weights</u> obtained from the respaective blocks ?</i>
 - <b>`1st MHA block` :</b> Self attention - Shows how each word from input sequence is related/affected by other words in the sentence. The attention weights obtained are multiplied with the `Value` vector to <b>emphasize the important features</b> of the encoder input i.e. <b>The intuition here is to keep intact the values of the word(s) we want to focus on, and drown-out irrelevant words</b>
 - <b>`2nd MHA block` :</b> Self attention - Shows how each word from output sequence is related/affected by other words in the sentence. The attention weights obtained are multiplied with the `Value` vector to <b>emphasize the important features</b> of the encoder input i.e. <b>The intuition here is to keep intact the values of the word(s) we want to focus on, and drown-out irrelevant words</b>
 - <b>`3rd MHA block` :</b> Context Attention - The attention weights obtained here will be used for visualization. Because only this shows, in what way each word of the decoder (output sequence, i.e. translated word) is related/affected to each word from the encoder (input sequence).

<i>If you are curious (or) want to check how the model performs, then you can save the attention weights obtained from the 1st and 2nd block to visualise the `Self Attention` of each word in a sentence.</i>

# 4. Residual Connections:

- The multi-headed attention block's output vector is added to the original positional input embedding. This is called a residual connection. The output of the residual connection then goes through a Normalization Layer.
- <u>There is a high possibility that the position information will get vanished once it reaches the upper layers</u> (i.e. once the input passes Linear and Multi-haed Attention Layers). So in order to safely maintain the positional information of the vectors we add the original input (which contains the positional information) of each multi-headed attention blocks to the output of the respective multi-headed attention blocks 😊


- During experimentation, the network displayed catastrophic results on removing the Residual Connections.
- <b>[Article](https://medium.com/@adityathiruvengadam/transformer-architecture-attention-is-all-you-need-aeccd9f50d09):</b> The authors have also discussed concatenation of the positional embeddings instead of adding them (ref: Allen NLP podcast). They were in the process of doing said experiments, but their initial results seem to say that the residual connections there can be mainly applied to the concatenated positional encoding section to propagate it through.

# 5. Dropout (DropAttention):

- In this paper, we demonstrate the benefit of dropout in self-attention layers (DropAttention) with two key distinctions compared with the dropout used in fully-connected layers and recurrent layers. The first is that DropAttention randomly sets attention weights to zero, which can be interpreted as dropping a set of neurons along different dimensions.
- <b>DropAttention aims to encourage the model to utilize the full context of the input sequences rather than relying solely on a small piece of features.</b>
- For example, for sentiment classification, the prediction is usually dominated by one or several emotional words, ignoring other informative patterns. This can make the model overfit some specific patterns.


- In fully-connected and recurrent layers, `Dropout` discourages the complex co-adaptation of different units in the same layer, while `DropAttention` prevents different contextualized feature vectors from co-adapting, learning features which are generally helpful for task-specific prediction. 
- Experiments on a wide range of tasks with different-scale datasets show that DropAttention can improve performance and reduce overfitting.

<b>Reference: [link](https://www.groundai.com/project/dropattention-a-regularization-method-for-fully-connected-self-attention-networks/1)</b>

## How stacked layers of Encoder and decoder work:
- The output of the 1st Encoder block is used as the input (Query, Key and Value) of the 2nd Encoder block and so on. 
- The same thing happens for the 1st Multi-head Attention of every decoder part present in the Decoder satck.
- And for the 2nd Multi-head Attention part of every decoder in the Decoder stack, the final output of the last encoder of the Encoder stack is used as `Key` and `Value`, and the output of it's corresponding 1st Multi-head Attention is used as `Query`.


- <b><i>Just like we did for simple encoder-decoder systems, we use the Encoder output (Key) and the Decoder output (Query) for calculating the attention weights. These attention weights are multiplied with the Encoder output to obtain the `context vector`.</i></b>

<img style="float: left;" src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/06/Screenshot-from-2019-06-17-20-03-14.png">

# Miscellaneous:

### <i>LayerNormalization</i>
- The layer normalizations are used to stabilize the network which results in substantially reducing the training time necessary. 
- Types and Uses of Normalization explained - https://medium.com/techspace-usict/normalization-techniques-in-deep-neural-networks-9121bf100d8


- <b><u><i>Point-wise Feedforward Layer</i></u> :</b> The pointwise feedforward layer is used to project the attention outputs potentially giving it a richer representation.

### <i>Why scaling is done in scaled dot-product?</i>
- The two most commonly used attention functions are additive attention and dot-product (multiplicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor
of `1/√dk`.
- Additive attention computes the compatibility function using a feed-forward network with a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is much faster and more space-efficient in practice, since it can be implemented using highly optimized matrix multiplication code.
- While for small values of dk the two mechanisms perform similarly, <u>additive attention outperforms
dot product attention without scaling for larger values of dk</u>. 

    <b><i>" We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has
extremely small gradients. " </b></i>


- <b><i>To counteract this effect, we scale the dot products by `1/√dk`</i></b>

# <u><i>Summary</i></u>:
#### Inputs to the self-attention module:
- Embedding module
- Positional encoding
- Truncating
- Masking

#### Adding more self-attention modules:
- Multihead
- Layer stacking

#### Modules between self-attention modules:
- Linear transformations 
- LayerNorm


# Limitations of Transformers:

- https://www.analyticsvidhya.com/blog/2019/06/understanding-transformers-nlp-state-of-the-art-models/
- https://medium.com/towards-artificial-intelligence/address-limitation-of-rnn-in-nlp-problems-by-using-transformer-xl-866d7ce1c8f4
- https://www.quora.com/What-are-the-benefits-of-Transformers-over-LSTMs

#### We want to translate a French sentence without having the German sentence:
The trick here is to re-feed our model for each position of the output sequence until we come across an end-of-sentence token.
A more step by step method would be:
Input the full encoder sequence (French sentence) and as decoder input, we take an empty sequence with only a start-of-sentence token on the first position. This will output a sequence where we will only take the first element.
That element will be filled into second position of our decoder input sequence, which now has a start-of-sentence token and a first word/character in it.
Input both the encoder sequence and the new decoder sequence into the model. Take the second element of the output and put it into the decoder input sequence.
Repeat this until you predict an end-of-sentence token, which marks the end of the translation.
We see that we need multiple runs through our model to translate our sentence.

## Use of self-attention:
- ”The animal didn't cross the street because it was too tired”

- What does “it” in this sentence refer to? Is it referring to the street or to the animal? It’s a simple question to a human, but not as simple to an algorithm.

- When the model is processing the word “it”, self-attention allows it to associate “it” with “animal”.

- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can <b>help lead to a better encoding for this word.

- Here we begin to see one key property of the Transformer, which is that the word in each position flows through its own path in the encoder. There are dependencies between these paths only in the self-attention layer. The feed-forward layer does not have those dependencies, however, and thus the various paths can be executed in parallel while flowing through the feed-forward layer (i.e., each word in a sequence do not interact with each other while travelling through the layers model of the model except in the self-attention layer - so we can compute the weights corresponding to each of these words induvidualy thus making  parallel computation possible)
- 

## Doubts:
- one fo LSTM's limitation is that Transfer Learning never really worked.Unlike ConvNets, you cant train on a different dataset and use it for your problem statement. You have to train on a specific dataset every time.

- In lstms (takes O(n^2) time) we can predict the next word only after predicting the previous word - so parallel computation is impossible (using gpu: 'Am i a joke to you?') - But with transformers you can do parallel computation - Even though the time required is O(n^2) for transformers, this time can be saved by using a gpu coz the input words can be passed in parallel unlike LSTM (input words are passed one by one).
 
- with transformers - transfer learning is possible
  
- When LSTM over transformers?: when sentence length is long or infinite coz transformers take N^2 time.
 
- Implementation code - hugging face - pre-trained transformer library

- Attention weights - how are they used - (nem_heads x att_wts) ??

- Cant we use the same LayerNorm everywhere - shoud they be unique??



- <b>Functions & Uses of the Encoder layer: </b>All the operations of the Encoder Layer are to encode the input to a continuous representation with attention information. This will help the decoder focus on the appropriate words in the input during the decoding process. <u>You can stack the encoder N times to further encode the information, where each layer has the opportunity to learn different attention representations therefore potentially boosting the predictive power of the transformer network</u>.

- The output of the first multi-headed attention is a masked output vector with information on how the model should attend on the decoder's input.

# <font color='darkblue'><u><i>Sample Example</i></u>:
<i>Sentiment Analysis using a pretrained model pipeline from hugging face library...</i>

In [1]:
import tensorflow
from transformers import pipeline

# You need internet connectivity to run this print statement (dont worry, consumes neglible amount of data only...)
classifier = pipeline('sentiment-analysis')


If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [2]:
print(classifier('I hate you'))

[{'label': 'NEGATIVE', 'score': 0.9991129040718079}]


In [3]:
print(classifier('I saw the movie bad boys for life'))

[{'label': 'NEGATIVE', 'score': 0.9738867878913879}]


In [4]:
print(classifier('I love you'))

[{'label': 'POSITIVE', 'score': 0.9998656511306763}]


In [5]:
print(classifier('did you go to work'))

[{'label': 'NEGATIVE', 'score': 0.5864599347114563}]
